<a href="https://colab.research.google.com/github/EmanoneL/data_analysis_pet_project/blob/main/CV_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install onnxruntime numpy opencv-python


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 94.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.2 MB/s eta 0:00:00


In [ ]:
# Отображение результатов
def draw_boxes(image, results, new_path):

    class_names = ['pleased', 'angry']
    height, width, _ = image.shape


    for box, score, class_id in results:
        # Корректируем координаты
        x1 = max(0, box[0])
        y1 = max(0, box[1])
        x2 = min(width, box[2])
        y2 = min(height, box[3])


        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)

        # Формируе метку с классом
        class_name = class_names[class_id]
        label = f"{class_name}: {score:.2f}"

        # Рассчитываем размеры текста
        font_scale = 0.5
        thickness = 1
        font = cv2.FONT_HERSHEY_SIMPLEX
        text_size = cv2.getTextSize(label, font, font_scale, thickness)[0]

        text_x, text_y = x1, y1 - 10
        text_x = max(0, min(width - text_size[0], text_x))  # Корректируем x
        text_y = max(text_size[1], text_y)  # Корректируем y


        cv2.putText(image, label, (text_x, text_y), font, font_scale, (0, 0, 0), thickness)

    cv2.imwrite(new_path, image)
    #cv2.imshow("Detections", image)


# def draw_boxes(image, results, class_names):
#     height, width, _ = image.shape

#     for box, confidence, class_id in results:
#         # Корректируем координаты
#         x1 = max(0, box[0])
#         y1 = max(0, box[1])
#         x2 = min(width, box[2])
#         y2 = min(height, box[3])

#         # Рисуем бокс
#         cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)

#         # Формируем метку с текстом
#         class_name = class_names[class_id]
#         label = f"{class_name}: {confidence:.2f}"

#         # Рассчитываем размеры текста
#         font_scale = 0.5
#         thickness = 1
#         font = cv2.FONT_HERSHEY_SIMPLEX
#         text_size = cv2.getTextSize(label, font, font_scale, thickness)[0]

#         text_x, text_y = x1, y1 - 10
#         text_x = max(0, min(width - text_size[0], text_x))  # Корректируем x
#         text_y = max(text_size[1], text_y)  # Корректируем y

#         # Рисуем фон для текста
#         cv2.rectangle(image, (text_x, text_y - text_size[1] - 2),
#                       (text_x + text_size[0], text_y + 2), (0, 255, 0), -1)

#         # Накладываем текст
#         cv2.putText(image, label, (text_x, text_y), font, font_scale, (0, 0, 0), thickness)


In [ ]:
def postprocess(output, confidence_threshold=0.1, nms_threshold=0.1):
    detections = output[0]  # Убираем пакетный размер
    num_predictions = detections.shape[2]

    boxes, confidences, class_ids = [], [], []

    for i in range(num_predictions):
        detection = detections[:, :, i].flatten()
        x, y, w, h = detection[0:4]
        confidence = detection[4]
        class_scores = detection[5:]

        if confidence < confidence_threshold:
            continue

        class_id = np.argmax(class_scores)
        class_score = class_scores[class_id]

        if class_score > confidence_threshold:
            x1 = int(x - w / 2)
            y1 = int(y - h / 2)
            x2 = int(x + w / 2)
            y2 = int(y + h / 2)

            print(f"Valid box: [{x1}, {y1}, {x2}, {y2}], confidence={confidence}, class_id={class_id}")

            boxes.append([x1, y1, x2, y2])
            confidences.append(float(confidence))
            class_ids.append(class_id)

    indices = cv2.dnn.NMSBoxes(boxes, confidences, confidence_threshold, nms_threshold)
    results = []

    if len(indices) > 0:
        for i in indices.flatten():
            results.append((boxes[i], confidences[i], class_ids[i]))

    return results


In [ ]:
# Функция для подготовки изображения
def preprocess_image(image_path, input_size):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    resized = cv2.resize(image, (input_size, input_size))
    # Преобразование в формат NCHW и нормализация
    image_data = resized.transpose((2, 0, 1)) / 255.0
    image_data = np.expand_dims(image_data, axis=0).astype(np.float32)
    return image, image_data


In [ ]:
import onnxruntime as ort
import numpy as np
import cv2

# Загрузка модели YOLOv11
model_path = "best.onnx"
session = ort.InferenceSession(model_path)

# Вывод информации о входах и выходах модели
print("Model inputs:", session.get_inputs())
print("Model outputs:", session.get_outputs())


Model inputs: [<onnxruntime.capi.onnxruntime_pybind11_state.NodeArg object at 0x7d4e71f8dd70>]
Model outputs: [<onnxruntime.capi.onnxruntime_pybind11_state.NodeArg object at 0x7d4e71f8fef0>]


In [ ]:
# Подготовка изображения
input_size = 640  # Размер входа модели (зависит от обучения)
image, image_data = preprocess_image("test3.jpg", input_size)

# Выполнение модели
input_name = session.get_inputs()[0].name
output = session.run(None, {input_name: image_data})

# Вывод результатов
for i, out in enumerate(output):
    print(f"Output[{i}] shape: {np.array(out).shape}")


Output[0] shape: (1, 6, 8400)


In [ ]:
results = postprocess(output)
results

[]

In [ ]:
draw_boxes(image, results, 'result.jpg')